In [ ]:
###############################################
# Author: Pedro Igor Salvador Alves
# Topic: Machine Learning with PySpark
# Start Date: 04/01/2022
# Type: Data Science - Machine Learning
###############################################

In [25]:
from pyspark.sql import SparkSession

In [26]:
spark = SparkSession.builder.appName("lr_pyspark_project").getOrCreate()

In [27]:
from pyspark.ml.regression import LinearRegression

In [28]:
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True, header=True)

In [29]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [30]:
for attribute in data.head(1)[0]:
    print(attribute)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [31]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [32]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [36]:
assembler = VectorAssembler(inputCols=(['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'])
                            ,outputCol='features')

In [39]:
output = assembler.transform(data)

In [40]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [41]:
final_data = output.select('features', 'Yearly Amount Spent')

In [42]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [43]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [44]:
lr = LinearRegression(labelCol="Yearly Amount Spent")

In [45]:
lr_model = lr.fit(train_data)

In [47]:
test_results = lr_model.evaluate(test_data)

In [49]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  9.542030436162804|
| -17.34518665000212|
|  -4.10201830150271|
| 6.6134015302252465|
| 3.6797858230044085|
|-13.213156909732277|
| -7.770908852838147|
|-3.7608504719630673|
| 2.8743904748495197|
|   2.41086485425393|
|  -8.35693094778992|
| -9.637368821280404|
| 3.0895607164405305|
|-17.641039498386817|
|-14.446395665217096|
|  -2.20968998292949|
|-1.0816051967470912|
| -7.700784438399182|
| -11.26610619674807|
| 0.8024536249702123|
+-------------------+
only showing top 20 rows



In [50]:
test_results.rootMeanSquaredError

9.206749646368948

In [51]:
test_results.r2

0.9856103238858793

In [52]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+

